### Problem 2

In [1]:
%%file reaction_coeffs.py
def k_const(k):
    """Returns the constant reaction rate coefficients
    
    INPUTS
    ======
    k: float, constant reaction rate coefficient
    
    RETURNS
    =======
    constant reaction rate coefficients, which is k
    
    EXAMPLES
    ========
    >>> k_const(1.5)
    1.5
    """
    k1 = k
    return k1

def k_arr(A, E, T, R=8.314):
    """Returns Arrhenius reaction rate coefficients
    
    INPUTS
    ======
    A: float, the Arrhenius prefactor, strictly positive
    E: float, a parameter
    T: float, temperature using a Kelvin scale, strictly positive
    R: float, default value is 8.314 which is the ideal gas constant. 
        R should not be changed except to convert units
    
    RETURNS
    =======
    Arrhenius reaction rate coefficients: a float
        a ValueError exception is raised if A <= 0
        a ValueError exception is raised if T <= 0
    Raise OverflowError if the result is beyond the maximum representable finite float
    Raise UnderflowError if the result is below the minimum positive normalized float
        since A, T and exponential function are all strictly positive,
        only consider positive overflow and positive underflow
    
    EXAMPLES
    ========
    >>>k_arr(10**7, 10**3, 10**2)
    3003549.08896
    """
    import numpy as np
    import sys
    if A <= 0:
        raise ValueError("The Arrhenius prefactor must be strictly positive.")
    if T <= 0:
        raise ValueError("Temperature must be strictly positive.")
    k2 = A * np.exp(-E / (R*T))
    if k2 > sys.float_info.max:
        raise OverflowError("Arrhenius coefficients overflow.")
    if k2 < sys.float_info.min:
        raise UnderflowError("Arrhenius coefficients underflow.")
    return k2

def k_modarr(A, b, E, T, R=8.314):
    """Returns the modified Arrhenius reaction rate coefficients.
    
    INPUTS
    ======
    A: float, the Arrhenius prefactor, strictly positive
    b: float, a parameter, must be real
    E: float, a parameter
    T: float, temperature using a Kelvin scale, strictly positive
    R: float, default value is 8.314 which is the ideal gas constant. 
        R should not be changed except to convert units
    
    RETURNS
    =======
    Modified Arrhenius reaction rate coefficients: a float
        a ValueError exception is raised if A <= 0
        a ValueError exception is raised if T <= 0
        a ValueError exception is raised if b is not real.
    Raise OverflowError if the result is beyond the maximum representable finite float
    Raise UnderflowError if the result is below the minimum positive normalized float
        since A, T and exponential function are all strictly positive,
        only consider positive overflow and positive underflow
    
    EXAMPLES
    ========
    >>>k_modarr(10**7, 0.5, 10**3, 10**2)
    30035490.889639609
    """
    import numpy as np
    import sys
    if A <= 0:
        raise ValueError("The Arrhenius prefactor must be strictly positive.")
    if T <= 0:
        raise ValueError("Temperature must be strictly positive.")
    if np.iscomplex(b):
        raise ValueError("The parameter b must be real.")
    k3 = A * (T**b) * np.exp(-E / (R*T))
    if k3 > sys.float_info.max:
        raise OverflowError("Arrhenius coefficients overflow.")
    if k3 < sys.float_info.min:
        raise UnderflowError("Arrhenius coefficients underflow.")
    return k3

Overwriting reaction_coeffs.py


### Problem 3

In [2]:
def progress_rate1(v_i, x, k):
    """Returns the progress rate for a reaction (v_A)A+(v_B)B --> (v_C)C
    
    INPUTS
    ====== 
    v_i: a list of floats representing Stoichiometric coefficients of reactants
        ordered according to the reaction form, left to right
    x: a list of floats representing the concentration of each reactant and product
        ordered according to the reaction form, left to right
    k: float, reaction rate coefficient
        
    RETURNS
    =======
    progress rate for the reaction: a float
    
    EXAMPLES
    ========
    >>>progress_rate1([2.0, 1.0, 0.0], [1.0, 2.0, 3.0], 10)
    20.0
    """
    r=1
    for v, xi in zip(v_i, x):
        r *= xi**v
    w = k*r
    return w

In [3]:
progress_rate1([2.0,1.0,0.0], [1.0,2.0,3.0], 10)

20.0

### Problem 4

In [4]:
def progress_rate2(v_ij1, v_ij2, x, k):
    """Returns the progress rate for a system of reactions of the following form:
        (v_11')A + (v_21')B --> (v_31'')C
        (v_12')A + (v_32')C --> (v_22'')B + (v_32'')C

    INPUTS
    =======
    v_ij1: a numpy array of floats, representing the stoichiometric coefficient of reactnat i in reaction j
    v_ij2: a numpy array of floats, representing the stoichiometric coefficient of product i in reactio j
    x: a list of floats representing the concentration of each reactant and product
    k: a list of floats, reaction rate coefficient for each reaction
    
    RETURNS
    =======
    progress rate for the system of reactions: a list of floats [w1, w2, ..., wj]
        wj is the progress rate for reaction j
    
    EXAMPLES:
    >>> progress_rate2(np.array([[1.0, 2.0, 0.0],[2.0, 0.0, 2.0]]),np.array([[0.0, 0.0, 2.0],
                        [0.0, 1.0, 1.0]]), [1.0,2.0,1.0], 10)
    [40.0, 10.0]

"""
    import numpy as np
    w_list = []
    each_reaction = np.vsplit(v_ij1,1)[0].tolist()
    for n in range(0, len(each_reaction)):
        r = 1
        for v, xi, ki in zip(each_reaction[n], x, k):
            r *= (xi**v)
            w = ki * r
        w_list.append(w)
    return w_list

In [5]:
import numpy as np
v_ij1 = np.array([[1.0, 2.0, 0.0], 
                 [2.0, 0.0, 2.0]])
v_ij2 = np.array([[0.0, 0.0, 2.0], 
                 [0.0, 1.0, 1.0]])
v_i1 = v_ij1[:][0]
v_i2 = v_ij1[:][1]
progress_rate2(v_ij1, v_ij2, [1.0,2.0,1.0], [10,10])

[40.0, 10.0]

In [6]:
np.vsplit(v_ij1, 1)[0].tolist()

[[1.0, 2.0, 0.0], [2.0, 0.0, 2.0]]

### Problem 5

In [7]:
def reaction_rate(v_ij1, v_ij2, x, k):
    """Returns the reaction rate for a system of reactions of the following form:
        (v_11')A + (v_21')B --> (v_31'')C
        (v_32')C --> (v_12'')A + (v_22'')B 
        
    INPUTS
    =======
    v_ij1: a numpy array of floats, representing the stoichiometric coefficient of reactnat i in reaction j
    v_ij2: a numpy array of floats, representing the stoichiometric coefficient of product i in reactio j
    x: a list of floats representing the concentration of each reactant and product
    k: float, reaction rate coefficient
    
    RETURNS
    =======
    reaction rate for the system of reactions: a list of floats [f1, f2, ..., fi]
        fj is the reaction rate for specie i
    
    EXAMPLES:
    >>> reaction_rate(np.array([[1.0, 2.0, 0.0], [0.0, 0.0, 2.0]]), np.array([[0.0, 0.0, 1.0], [1.0, 2.0, 0.0]]),
                      [1.0,2.0,1.0], 10)
    [-30.0, -60.0, 20.0]

"""
    import numpy as np
    v_ij = v_ij2 - v_ij1
    f_list = []
    if v_ij.ndim > 1:
        w_list = progress_rate2(v_ij1, v_ij2, x, k)
        component = np.vsplit(np.transpose(v_ij),1)[0].tolist()
        for n in range(0, len(component)):
            f = 0
            for i in range(0, len(w_list)):
                f += component[n][i] * w_list[i]
            f_list.append(f)
    
    if v_ij.ndim == 1:
        w_list = progress_rate1(v_ij1, x, k)
        component = v_ij.tolist()
        for i in range(0, len(w_list)):
            f_list.append(v_ij[i] * w_list[i])
    return f_list

In [8]:
v_ij1 = np.array([[1.0, 2.0, 0.0], 
                 [0.0, 0.0, 2.0]])
v_ij2 = np.array([[0.0, 0.0, 1.0], 
                 [1.0, 2.0, 0.0]])

reaction_rate(v_ij1, v_ij2, [1.0,2.0,1.0], [10, 10])

[-30.0, -60.0, 20.0]

### Problem 6

In [9]:
%%file chemkin.py
def progress_rate1(v_i, x, k):
    """Returns the progress rate for a reaction (v_A)A+(v_B)B --> (v_C)C
    
    INPUTS
    ====== 
    v_i: a list of floats representing Stoichiometric coefficients of reactants
        ordered according to the reaction form, left to right
    x: a list of floats representing the concentration of each reactant and product
        ordered according to the reaction form, left to right
    k: float, reaction rate coefficient
        
    RETURNS
    =======
    progress rate for the reaction: a float
    
    EXAMPLES
    ========
    >>>progress_rate1([2.0, 1.0, 0.0], [1.0, 2.0, 3.0], 10)
    20.0
    """
    r=1
    for v, xi in zip(v_i, x):
        r *= xi**v
    w = k*r
    return w

def progress_rate2(v_ij1, v_ij2, x, k):
    """Returns the progress rate for a system of reactions of the following form:
        (v_11')A + (v_21')B --> (v_31'')C
        (v_12')A + (v_32')C --> (v_22'')B + (v_32'')C

    INPUTS
    =======
    v_ij1: a numpy array of floats, representing the stoichiometric coefficient of reactnat i in reaction j
    v_ij2: a numpy array of floats, representing the stoichiometric coefficient of product i in reactio j
    x: a list of floats representing the concentration of each reactant and product
    k: float, reaction rate coefficient
    
    RETURNS
    =======
    progress rate for the system of reactions: a list of floats [w1, w2, ..., wj]
        wj is the progress rate for reaction j
    
    EXAMPLES:
    >>> progress_rate2(np.array([[1.0, 2.0, 0.0],[2.0, 0.0, 2.0]]),np.array([[0.0, 0.0, 2.0],
                        [0.0, 1.0, 1.0]]), [1.0,2.0,1.0], 10)
    [40.0, 10.0]

"""
    import numpy as np
    w_list = []
    each_reaction = np.vsplit(v_ij1,1)[0].tolist()
    for n in range(0, len(each_reaction)):
        r = 1
        for v, xi, ki in zip(each_reaction[n], x, k):
            r *= (xi**v)
            w = ki * r
        w_list.append(w)
    return w_list

#     import numpy as np
#     w_list = []
#     if v_ij1.ndim > 1:
#         each_reaction = np.vsplit(v_ij1,1)[0].tolist()
#         for n in range(0, len(each_reaction)):
#             r = 1
#             for v, xi in zip(each_reaction[n], x):
#                 r *= xi**v
#             w = k*r
#             w_list.append(w)
        
#     if v_ij1.ndim == 1:
#         w_list.append(progress_rate1(v_ij1, x, k))
#     return w_list

def reaction_rate(v_ij1, v_ij2, x, k):
    """Returns the reaction rate for a system of reactions of the following form:
        (v_11')A + (v_21')B --> (v_31'')C
        (v_32')C --> (v_12'')A + (v_22'')B 
        
    INPUTS
    =======
    v_ij1: a numpy array of floats, representing the stoichiometric coefficient of reactnat i in reaction j
    v_ij2: a numpy array of floats, representing the stoichiometric coefficient of product i in reactio j
    x: a list of floats representing the concentration of each reactant and product
    k: a list of floats, reaction rate coefficient for each reaction
    
    RETURNS
    =======
    reaction rate for the system of reactions: a list of floats [f1, f2, ..., fi]
        fj is the reaction rate for specie i
    
    EXAMPLES:
    >>> reaction_rate(np.array([[1.0, 2.0, 0.0], [0.0, 0.0, 2.0]]), np.array([[0.0, 0.0, 1.0], [1.0, 2.0, 0.0]]),
                      [1.0,2.0,1.0], 10)
    [-30.0, -60.0, 20.0]

"""
#     import numpy as np
#     v_ij = v_ij2 - v_ij1
#     if v_ij.ndim > 1:
#         component = np.vsplit(np.transpose(v_ij),1)[0].tolist()
#     if v_ij.ndim == 1:
#         component = v_ij.tolist()
#     f_list = []
#     w_list = progress_rate2(v_ij1, v_ij2, x, k)
#     for n in range(0, len(component)):
#         f = 0
#         for i in range(0, len(w_list)):
#             f += component[n][i] * w_list[i]
#         f_list.append(f)
#     return f_list

    import numpy as np
    v_ij = v_ij2 - v_ij1
    f_list = []
    if v_ij.ndim > 1:
        w_list = progress_rate2(v_ij1, v_ij2, x, k)
        component = np.vsplit(np.transpose(v_ij),1)[0].tolist()
        for n in range(0, len(component)):
            f = 0
            for i in range(0, len(w_list)):
                f += component[n][i] * w_list[i]
            f_list.append(f)
    
    if v_ij.ndim == 1:
        w_list = progress_rate1(v_ij1, x, k)
        component = v_ij.tolist()
        for i in range(0, len(w_list)):
            f_list.append(v_ij[i] * w_list[i])
    return f_list

Overwriting chemkin.py


In [10]:
import reaction_coeffs
import chemkin

T = [750, 1500, 2500]
k_reaction1 = [reaction_coeffs.k_modarr(10**8, 0.5, 5*(10**4), t) for t in T]
k_reaction2 = [reaction_coeffs.k_const(10**4) for t in T]
k_reaction3 = [reaction_coeffs.k_arr(10**7, 10**4, t) for t in T]

k_750 = [k_reaction1[0], k_reaction2[0], k_reaction3[0]]
k_1500 = [k_reaction1[1], k_reaction2[1], k_reaction3[1]]
k_2500 = [k_reaction1[2], k_reaction2[2], k_reaction3[2]]


# reaction 1: 2(H2) + O2 -> 2(OH) + H2
# reaction 2: OH + HO2 -> H2O + O2
# reaction 3: H2O + O2 -> HO2 + OH
v_p = np.array([[2.0, 1.0, 0.0, 0.0, 0.0], 
                [0.0, 0.0, 1.0, 1.0, 0.0],
                [0.0, 1.0, 0.0, 0.0, 1.0]])

v_pp = np.array([[1.0, 0.0, 2.0, 0.0, 0.0], 
                 [0.0, 1.0, 0.0, 0.0, 1.0],
                 [0.0, 0.0, 1.0, 1.0, 0.0]])

x = [2.0, 1.0, 0.5, 1.0, 1.0] # H2 O2 OH HO2 H2O

In [14]:
reaction_rate_750 = [chemkin.reaction_rate(v_p, v_pp, x, k_750)]
reaction_rate_1500 = [chemkin.reaction_rate(v_p, v_pp, x, k_1500)]
reaction_rate_2500 = [chemkin.reaction_rate(v_p, v_pp, x, k_2500)]

print("The reaction rates of the five species H2, O2, OH, HO2 and H2O at T=750K is:")
for i in range(0, len(reaction_rate_750)):
    print(reaction_rate_750[i])
    
print("The reaction rates of the five species H2, O2, OH, HO2 and H2O at T=1500K is:")
for i in range(0, len(reaction_rate_1500)):
    print(reaction_rate_1500[i])
          
print("The reaction rates of the five species H2, O2, OH, HO2 and H2O at T=2500K is:")
for i in range(0, len(reaction_rate_2500)):
    print(reaction_rate_2500[i])

The reaction rates of the five species H2, O2, OH, HO2 and H2O at T=750K is:
[-8045869.2432669112, -9051602.898675276, 17097472.141942188, 1005733.6554083639, -1005733.6554083639]
The reaction rates of the five species H2, O2, OH, HO2 and H2O at T=1500K is:
[-17939753.892700881, -20182223.129288491, 38121977.021989368, 2242469.2365876101, -2242469.2365876101]
The reaction rates of the five species H2, O2, OH, HO2 and H2O at T=2500K is:
[-24723723.902628928, -27814189.390457544, 52537913.293086477, 3090465.4878286161, -3090465.4878286161]


In [16]:
%%file test_chemkin.py
import chemkin

def test_list_type():
    try:
        chemkin.progress_rate1([2.0, 1.0, 0.0], ["", "green", "hi"], 10)
    except TypeError as err:
        assert(type(err) == TypeError)

Overwriting test_chemkin.py


# IGNORE BELOW

In [ ]:
# test
def k_arr(A, E, T, R=8.314):
    import numpy as np
    if A <= 0:
        raise ValueError("The Arrhenius prefactor must be strictly positive.")
    if T <= 0:
        raise ValueError("Temperature must be strictly positive.")
    k2 = A * np.exp(-E / (R*T))
    return k2

In [ ]:
k2 = k_arr(10**7, 10**3, 10**2)
print(k2)

In [ ]:
def k_modarr(A, b, E, T, R=8.314):
    import numpy as np
    if A <= 0:
        raise ValueError("The Arrhenius prefactor must be strictly positive.")
    if T <= 0:
        raise ValueError("Temperature must be strictly positive.")
    if np.iscomplex(b):
        raise ValueError("The parameter b must be real.")
    k3 = A * (T**b) * np.exp(-E / (R*T))
    return k3

In [ ]:
k_modarr(10**7, 0.5, 10**3, 10**2)

In [ ]:
import sys
print(sys.float_info.max)
print(sys.float_info.min)